## 2. Define the hyperparameters

Some hyperparameters are entered by default, for example the Gaussian process kernel and the multi-fidelity acquisition function.

In [ ]:
dim = 1
iterations = 10
seed = 123
samp_nos = [5, 2] # no. of initial DoE points
fids = [0.5, 1.] # add comments... 
costs = [0.5, 1.]

### 3. Specify the multi-fidelity problem

- This means that the functions or models that represent the different fidelities need to be defined and collected into a multi-fidelity function. 

- The design space corresponding to this multi-fidelity function is also needed, in order to define a sampler to sample initial training data from.

The review manuscript works with augmented analytical functions to create the basis of a multi-fidelity problem. 

This starts by picking a base function.

In [ ]:
base_fun = f3dasm.functions.AlpineN2(
    dimensionality=dim,
    scale_bounds=np.tile([0.0, 1.0], (dim, 1)),
    )

Once the function has been defined, the pipeline function `create_analytical_mf_problem` can be used to generate the multi-fidelity assets needed to run the optimization problem.

In [ ]:
mf_train_data, mffun, _, mf_sampler = f3dasm.create_analytical_mf_problem(
    base_fun=base_fun,
    dim=dim,
    fids=fids,
    costs=costs,
    samp_nos=samp_nos,
    )

In [ ]:
mf_train_data[1]


Data(design=DesignSpace(input_space=[ContinuousParameter(name='x0', _type='float', lower_bound=0.0, upper_bound=1.0), ConstantParameter(name='fid', _type='float', constant_value=1.0)], output_space=[ContinuousParameter(name='y', _type='float', lower_bound=-inf, upper_bound=inf)]), data=  input        output
     x0  fid        y
0   0.0  1.0  0.00000
1   0.5  1.0  2.14422)

### 4. Define and initialize optimizer

This optimizer makes use of the variable-fidelity UCB acquisition function.

```
Jiang, P.; Cheng, J.; Zhou, Q.; Shu, L. & Hu, J.
Variable-fidelity lower confidence bounding approach for engineering optimization problems with expensive simulations
AIAA Journal, American Institute of Aeronautics and Astronautics, 2019, 57, 5416-5430
```

In [ ]:
optimizer = f3dasm.optimization.MFBayesianOptimizationTorch(
    data=mf_train_data,
    mffun=mffun,
)

optimizer.init_parameters()

### 5. Run optimization and obtain results

In [ ]:
res = f3dasm.run_mf_optimization(
    optimizer=optimizer,
    mffunction=mffun,
    sampler=mf_sampler[-1],# remove / adjust this!
    iterations=iterations,
    seed=123,
    number_of_samples=samp_nos,
)

# Add more options to run / define optimizers

iteration 0
[[0.78157627 0.5       ]] [[-1.3968135]]
iteration 1
[[0.8067155 0.5      ]] [[-1.38799132]]
iteration 2
[[0.81962762 0.5       ]] [[-1.34841381]]
iteration 3
[[0.82046568 0.5       ]] [[-1.34502702]]
iteration 4
[[0.82072046 0.5       ]] [[-1.34397777]]
iteration 5
[[0.82086617 0.5       ]] [[-1.34337355]]
iteration 6
[[0.82095456 0.5       ]] [[-1.34300558]]
iteration 7
[[0.82100717 0.5       ]] [[-1.34278607]]
iteration 8
[[0.82103197 0.5       ]] [[-1.34268242]]
iteration 9
[[0.82102681 0.5       ]] [[-1.34270401]]


In [ ]:

res[1].data

input         output
          x0  fid         y
0   0.000000  0.5  0.000000
1   0.500000  0.5  1.072110
2   0.750000  0.5 -1.284409
3   0.250000  0.5 -0.473134
4   0.375000  0.5  0.553412
5   0.000000  1.0  0.000000
6   0.500000  1.0  2.144220
7   0.781576  0.5 -1.396814
8   0.806716  0.5 -1.387991
9   0.819628  0.5 -1.348414
10  0.820466  0.5 -1.345027
11  0.820720  0.5 -1.343978
12  0.820866  0.5 -1.343374
13  0.820955  0.5 -1.343006
14  0.821007  0.5 -1.342786
15  0.821032  0.5 -1.342682
16  0.821027  0.5 -1.342704